In [10]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd 
import numpy as np 
import time, random
import re
import csv

In [11]:
header = {"User-Agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"}

In [12]:
# A txt file of California county names
with open("/Users/che-yuliu/Downloads/County_List_Cal.txt") as f:
    county = [line.strip() for line in f.readlines()]
int_list = range(303,361)

In [13]:
# This list will aid in getting the full URL for each county in California.
county_list = []
for i in range(len(county)):
    county_list.append(f"{int_list[i]}/CA/{county[i]}")
county_list

['303/CA/Alameda-County',
 '304/CA/Alpine-County',
 '305/CA/Amador-County',
 '306/CA/Butte-County',
 '307/CA/Calaveras-County',
 '308/CA/Colusa-County',
 '309/CA/Contra-Costa-County',
 '310/CA/Del-Norte-County',
 '311/CA/El-Dorado-County',
 '312/CA/Fresno-County',
 '313/CA/Glenn-County',
 '314/CA/Humboldt-County',
 '315/CA/Imperial-County',
 '316/CA/Inyo-County',
 '317/CA/Kern-County',
 '318/CA/Kings-County',
 '319/CA/Lake-County',
 '320/CA/Lassen-County',
 '321/CA/Los-Angeles-County',
 '322/CA/Madera-County',
 '323/CA/Marin-County',
 '324/CA/Mariposa-County',
 '325/CA/Mendocino-County',
 '326/CA/Merced-County',
 '327/CA/Modoc-County',
 '328/CA/Mono-County',
 '329/CA/Monterey-County',
 '330/CA/Napa-County',
 '331/CA/Nevada-County',
 '332/CA/Orange-County',
 '333/CA/Placer-County',
 '334/CA/Plumas-County',
 '335/CA/Riverside-County',
 '336/CA/Sacramento-County',
 '337/CA/San-Benito-County',
 '338/CA/San-Bernardino-County',
 '339/CA/San-Diego-County',
 '340/CA/San-Francisco-County',
 '34

In [28]:
# Testing to see 
url1= "https://www.redfin.com/county/303/CA/Alameda-County"

response = requests.get(url1, headers = header)
response.status_code

200

In [29]:
# Code idea obtained from Metis webscraping exercise 8 solutions
redfin_links = []

base_url = "https://www.redfin.com/county/"
url_list = [base_url + str(i) for i in county_list]

for index, url in enumerate(url_list):
    if index %5 == 0: 
        time.sleep(4+2*random.random())
    page = requests.get(url, headers = header).text
    soup = bs(page)

    for div in soup.find_all(class_ ="bottomV2"):
        base_link = 'https://www.redfin.com/'
        a_tag = div.find('a')
        a_link = a_tag['href']
        #Concatenates the base link to 
        h_links = base_link + a_link
        #Appends full links to a list 
        redfin_links.append(h_links)

In [31]:
with open('/Users/che-yuliu/Desktop/redfin_links.csv', 'w', encoding = "UTF8",newline = "") as z:
    quill = csv.writer(z)
    for link in redfin_links:
        
        quill.writerow([link])


In [136]:
listing_details = ["address", "price", "bedrooms", "baths", 
                                 "square_ft", "total_time_listed", "property_type", 
                                "year_built", "hoa_dues", "community", 
                                 "lot_size", "price/sqft"]
with open("/Users/che-yuliu/Desktop/listings.csv", "w", encoding = "UTF8",newline = "") as p:
    writer = csv.DictWriter(p, fieldnames = listing_details)
    writer.writeheader()
    writer.writerows(listing_info)
    

In [104]:
# This function serves to grab certain parts of an beautifulsoup object. 

def get_info(r, home_deets):
    
    find_tag = r.find_all(class_ = "keyDetail font-weight-roman font-size-base")
    for q in find_tag:
 
        find_span = q.find("span").text
      
        if find_span == home_deets:

            span2 = q.find("span").next.next.text
        
            return span2
    return None 




In [225]:
# Getting data from specific redfin housing listings and appending it to a csv file.

listing_info = []

for index, URL in enumerate(redfin_links[0:3]):
    if index %5 == 0:
        time.sleep(1+2*random.random())
    listing_tag = requests.get(URL, headers = header).text
    listing_page = bs(listing_tag)
    
    for r in listing_page.find_all(class_="alongTheRail"):
        try:
            #gets the address tag by looking for the class homeAddress
            address_tag = r.find(class_="homeAddress")
            #turns it in to a bs text 
            address = address_tag.text.split(",")[0]
            #gets the price tag  
            price_tag = r.find("div", class_="stat-block beds-section").next
            price = price_tag.text[1:].replace(",", "")
            #gets the bed tag
            bed_tag = r.find_all("div", class_="stat-block beds-section")[1]
            bed = bed_tag.text.split("B")[0]
            #gets bathroom tag
            bath_tag = r.find(class_ = "stat-block baths-section").next
            bath = bath_tag.text
            #gets square footage of house
            sqft_tag = r.find(class_ = "stat-block sqft-section").next
            sqft = sqft_tag.text.replace(",", "")
            # gets amount of time on redfin
            redtime = get_info(r, "Time on Redfin")
            # gets type of property
            proptype = get_info(r, "Property Type")
            # getting year built 
            yr_built = get_info(r, "Year Built")
            hoa_dues = get_info(r, "HOA Dues")
            community = get_info(r, "Community")
            lot_size = get_info(r, "Lot Size")
            price_sqft = get_info(r, "Price/Sq.Ft.")
            #appending the bs text into a dictionary -- later append it to an excel file or a csv
            listing_info.append({"address": address, "price": price, "bedrooms": bed, "baths" : bath, 
                                 "square_ft" : sqft, "total_time_listed" : redtime, "property_type" : proptype, 
                                "year_built" : yr_built, "hoa_dues" : hoa_dues, "community" : community, 
                                 "lot_size" : lot_size, "price/sqft" : price_sqft})
        except:
            print("nothing to assign")


In [217]:
with open("/Users/che-yuliu/Desktop/listings.csv", "a", encoding = "UTF8",newline = "") as p:
    writer = csv.DictWriter(p, fieldnames = listing_details)
    writer.writerows(listing_info)

In [226]:
df = pd.DataFrame(listing_info)
df

,address,price,bedrooms,baths,square_ft,total_time_listed,property_type,year_built,hoa_dues,community,lot_size,price/sqft
0,11643 Silvergate Dr,1690000,4,2,1888,None,None,None,None,None,None,None
1,7806 Peachtree Ave,1249888,3,2.5,1356,None,None,None,None,None,None,None
2,38746 Buckboard Cmn,1298000,3,2.5,1780,None,None,None,None,None,None,None
